In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 0)

node[0].set_service(jobclass[0], Erlang(3, 2))
node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))

node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))
node[1].set_service(jobclass[1], Exp(1))

P = model.init_routing_matrix()
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.1)
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)

P.set(jobclass[0], jobclass[1], node[0], node[0], 0.6)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)

P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

model.link(P)

In [3]:
# Aligned with JAR test scenarios for cqn_twoclass_hyperl
# JAR tests: CTMC(), JMT(seed=23000, samples=5000), SSA(seed=23000, samples=5000),
#           Fluid(), MVA(method="exact"), NC(method="exact"), MAM(), DES(seed=23000, samples=5000)

solver = np.array([], dtype=object)

# CTMC with default settings (matches JAR)
solver = np.append(solver, CTMC(model))

# JMT with seed=23000, samples=5000 (matches JAR)
solver = np.append(solver, JMT(model, seed=23000, samples=5000))

# SSA with seed=23000, samples=5000 (matches JAR)
solver = np.append(solver, SSA(model, seed=23000, samples=5000))

# Fluid with default settings (matches JAR)
solver = np.append(solver, FLD(model))

# MVA with method="exact" (matches JAR)
solver = np.append(solver, MVA(model, method='exact'))

# NC with method="exact" (matches JAR)
solver = np.append(solver, NC(model, method='exact'))

# MAM with default settings (matches JAR)
solver = np.append(solver, MAM(model))

# DES with seed=23000, samples=5000 (matches MATLAB)
solver = np.append(solver, DES(model, seed=23000, samples=5000))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()

\nSOLVER: CTMC
Station JobClass       QLen       Util  RespT     ResidT       ArvR       Tput
  Delay   Class1 1.3604e+00 1.3604e+00 0.6667 6.6667e-01 2.0406e+00 2.0406e+00
  Delay   Class2 1.2027e-17 1.2027e-17 0.2167 1.4920e-17 5.5511e-17 5.5511e-17
 Queue1   Class1 6.3962e-01 1.9385e-01 0.6269 3.1345e-01 1.0203e+00 1.0203e+00
 Queue1   Class2 2.0000e+00 8.0615e-01 2.4809 2.4809e+00 8.0615e-01 8.0615e-01
\nSOLVER: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.0808 1.0808 0.6619  0.6619 1.4927 1.4927
  Delay   Class2 0.1959 0.1959 0.2166  0.2166 0.9509 0.9509
 Queue1   Class1 0.0847 0.0305 0.5264  0.5264 0.1564 0.1564
 Queue1   Class2 2.6841 0.9385 2.9622  2.9622 0.9441 0.9441
\nSOLVER: SSA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.9236 0.9236 0.6364  0.6364 1.4513 1.4513
  Delay   Class2 0.2172 0.2172 0.2257  0.2257 0.9626 0.9626
 Queue1   Class1 0.0707 0.0241 0.5608  0.5608 0.1260 0.1260
 Queue1   Class2 2.7885 0.9522 2.9031  2.9031 0.9605 0.9605
\nSOLVER: Fluid
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.9537 0.9537 0.6667  0.6667 1.4306 1.4306
  Delay   Class2 0.2108 0.2108 0.2167  0.2167 0.9728 0.9728
 Queue1   Class1 0.0771 0.0272 0.5387  0.5387 0.1431 0.1431
 Queue1   Class2 2.7584 0.9728 2.8355  2.8355 0.9728 0.9728
\nSOLVER: MVA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   T

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.2581 1.2581 0.6687  0.6687 1.3945 1.3945
  Delay   Class2 0.3111 0.3111 0.2081  0.2081 0.9532 0.9525
 Queue1   Class1 0.2429 0.0274 2.0496  2.0496 0.1288 0.1288
 Queue1   Class2 2.1879 0.9469 2.7398  2.7398 0.9525 0.9504
